# Paper

#### Import Functions

In [ ]:
%matplotlib inline

#import pyvttbl as pt
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.api as sm

import statsmodels.formula.api as smf

import numpy as np
import sklearn
import pandas as pd
#import preprocess2
#import prepare
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from scipy import stats
import warnings
from scipy.stats.mstats import zscore
import matplotlib
#matplotlib.use('TkAgg')  
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as hac
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import cross_val_score
import mne
import imageio
import datetime as dt
import os 


#### Loading the Data

In [ ]:
df=pd.read_csv('./final_df_mb.csv', sep=',', na_values=['?']) 

## Participants (Demographics and Sample Characteristics)

In [ ]:
df=df[df.vpn!=26] # hier ist irgendetwas schief gegangen - die Zeiten stimmen nicht!! 
df=df[df.vpn!=33] # hier ist irgendetwas schief gegangen - die Zeiten stimmen nicht!! 
df=df[df.vpn!=34] # extrem unruhig - und Ausreißer-Werte bei LEV
df=df[df.vpn!=37] # extrem unruhig
df=df[df.conversation!='base'].reset_index() #delete the trigger and base

In [ ]:
df['participant_speaks']=(df.conversation=='joy_proband') | (df.conversation=='disgust_proband') | (df.conversation=='neutral_proband')#if conversation==
df=df[(df.participant_speaks==False)&(df.conversation!='intro')].reset_index(drop=True)

In [ ]:
df['ZYG_z']=scipy.stats.zscore(df['ZYG'])
df['COR_z']=scipy.stats.zscore(df['COR'])
df['LEV_z']=scipy.stats.zscore(df['LEV'])

In [ ]:
print ('number of particpants ' + str(len(df.groupby('vpn'))))
#print 'number of females ' + str(len(df[df.gender==1].groupby('vpn')))
print ('min age ' + str(np.min(df.groupby('vpn').mean().years)))
print ('max age ' + str(np.max(df.groupby('vpn').mean().years)))
print ('mean age ' + str(np.mean(df.groupby('vpn').mean().years)))
print ('std age ' + str(np.std(df.groupby('vpn').mean().years)))
print ('AQ above cut-off of 17: ' + str(np.sum(df.groupby('vpn').mean().AQsum>17)))
print ('mean AQ' + str(np.mean(df.groupby('vpn').mean().AQsum)))
print ('mean AQ' + str(np.std(df.groupby('vpn').mean().AQsum)))
print ('mind. high-school education: ' + str(np.sum(df.groupby('vpn').mean().education>5)))
print ('WST: ' + str(np.sum(df.groupby('vpn').mean().WST<20)))

## EMG Activity over Timecourse of Conversation

In [ ]:
colors=['blue', 'green','red']
#final=df[(df.vpn==34)]# | (df.vpn!=33)]
final=df
muscles=['ZYG_z', 'COR_z', 'LEV_z']

for conv in np.unique(final.conversation):
    i=0
    for m in muscles:
        plt.plot(np.array(final[(final['conversation']==conv)].groupby(['counter']).mean().reset_index()['counter']), 
                 np.array(final[(final['conversation']==conv)].groupby(['counter']).mean().reset_index(drop=True)[m]),
                 color=colors[i])
        plt.title(str(conv))
        #plt.ylim(0, 200)
        i=i+1
    plt.show()

In [ ]:
plt.hist(df.groupby('vpn').mean()['ZYG_z'])
plt.show()
plt.hist(df.groupby('vpn').mean()['COR_z'])
plt.show()
plt.hist(df.groupby('vpn').mean()['LEV_z'])
plt.show()

In [ ]:
df_figure=df.rename(columns={'ZYG_z': 'ZYG (zstd.)', 'COR_z': 'COR (zstd.)', 'LEV_z': 'LEV (zstd.)'})
di = {"neutral_speaker":"1.Table Setting",
      "joy_speaker":"2.Liked Food",
      "disgust_speaker":"3.Disliked Food"}
df_figure=df_figure.replace({"conversation": di})

In [ ]:
g=sns.heatmap(df_figure[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby('conversation').mean()[['ZYG (zstd.)', 'COR (zstd.)', 'LEV (zstd.)']], 
              cmap='Blues')##,  vmin=-0.3, vmax=0.4)
#cmap='RdBu_r',

#print (df[df.participant_speaks==True].groupby('conversation').mean()[['ZYG_z', 'COR_z']])

#plt.set
plt.xticks(fontsize=14) 
plt.yticks(fontsize=14) 
plt.xlabel('Muscle', fontsize=18) 
plt.ylabel('Conversation Part', fontsize=18)
plt.title('Participant Listening')
plt.savefig('Muscles_Listening_' +'.pdf', format='pdf', bbox_inches = "tight")
plt.savefig('Muscles_Listening_' +'.png', format='png', bbox_inches = "tight")
plt.tight_layout()
plt.show()   
plt.close()

In [ ]:

print (df_figure[(df_figure.participant_speaks==False) &(df_figure.conversation!='intro')].groupby('conversation').mean()[['ZYG (zstd.)', 'COR (zstd.)', 'LEV (zstd.)']])
print (df_figure[(df_figure.participant_speaks==False) &(df_figure.conversation!='intro')][['ZYG (zstd.)', 'COR (zstd.)', 'LEV (zstd.)', 'conversation']].groupby('conversation').std())


In [ ]:
df1=df[df.conversation=='neutral_speaker'].groupby('vpn').mean()
df2=df[df.conversation=='joy_speaker'].groupby('vpn').mean()
df3=df[df.conversation=='disgust_speaker'].groupby('vpn').mean()

In [ ]:
import mystats
mystats.two_dep_sample_tests(df1, df2, 'ZYG_z')
mystats.two_dep_sample_tests(df1, df3, 'ZYG_z')

mystats.two_dep_sample_tests(df1, df2, 'COR_z')
mystats.two_dep_sample_tests(df1, df3, 'COR_z')


In [ ]:

mystats.two_dep_sample_tests(df1, df2, 'LEV_z')
mystats.two_dep_sample_tests(df1, df3, 'LEV_z')
0.008

## Correlation of EMG and Questionnaire Values

In [ ]:
emotion=3
muscle=3

neutral_cor=df[df.conversation=='neutral_speaker'].groupby('vpn').mean()['COR_z']
neutral_zyg=df[df.conversation=='neutral_speaker'].groupby('vpn').mean()['ZYG_z']
neutral_lev=df[df.conversation=='neutral_speaker'].groupby('vpn').mean()['LEV_z']
         
pos_cor=df[df.conversation=='joy_speaker'].groupby('vpn').mean()['COR_z']   
pos_zyg=df[df.conversation=='joy_speaker'].groupby('vpn').mean()['ZYG_z']
pos_lev=df[df.conversation=='joy_speaker'].groupby('vpn').mean()['LEV_z']
         
neg_cor=df[df.conversation=='disgust_speaker'].groupby('vpn').mean()['COR_z']        
neg_zyg=df[df.conversation=='disgust_speaker'].groupby('vpn').mean()['ZYG_z']
neg_lev=df[df.conversation=='disgust_speaker'].groupby('vpn').mean()['LEV_z']

print ('Effect of Muscle and Emotion')
data=np.array(pd.concat([neutral_cor, neutral_zyg, neutral_lev, 
                         pos_cor, pos_zyg, pos_lev, 
                         neg_cor, neg_zyg, neg_lev], axis=1))
factorlevels=[emotion, muscle]

print (mne.stats.f_mway_rm(data, factorlevels, effects='A*B'))

In [ ]:
zyg=df[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby(['vpn']).mean()[['ZYG_z', 'AQsum']].reset_index()
zyg['muscle']='ZYG'
zyg['activity']=zyg.ZYG_z
cor=df[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby(['vpn']).mean()[['COR_z', 'AQsum']].reset_index()
cor['muscle']='COR'
cor['activity']=cor.COR_z
data=pd.concat([zyg, cor], sort=True)

model = sm.MixedLM.from_formula("activity ~ AQsum*muscle ", data[data.AQsum>0], groups=data['vpn'])
mdf = model.fit()
print(mdf.summary())

In [ ]:
zyg=df[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby(['vpn']).mean()[['ZYG_z', 'AQsum']].reset_index()
zyg['muscle']='ZYG'
zyg['activity']=zyg.ZYG_z
cor=df[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby(['vpn']).mean()[['COR_z', 'AQsum']].reset_index()
cor['muscle']='COR'
cor['activity']=cor.COR_z

lev=df[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby(['vpn']).mean()[['LEV_z', 'AQsum']].reset_index()
lev['muscle']='LEV'
lev['activity']=lev.LEV_z
data=pd.concat([zyg, cor, lev], sort=True)

model = sm.MixedLM.from_formula("activity ~ AQsum*muscle ", data[data.AQsum>0], groups=data['vpn'])
mdf = model.fit()
print(mdf.summary())

In [ ]:
data=df[(df_figure.participant_speaks==False) &(df.conversation!='intro')].groupby(['conversation', 'vpn']).mean().reset_index()

model = sm.MixedLM.from_formula("ZYG_z ~ AQsum", data, groups=data['vpn'])
mdf = model.fit()
print(mdf.summary())


model = sm.MixedLM.from_formula("COR_z ~ AQsum", data, groups=data['vpn'])
mdf = model.fit()
print(mdf.summary())


model = sm.MixedLM.from_formula("LEV_z ~ AQsum", data, groups=data['vpn'])
mdf = model.fit()
print(mdf.summary())


## Correlation of Food Preferences and EMG actvity

In [ ]:
plt.scatter(df[df.conversation=='joy_speaker'].groupby('vpn').mean()['ZYG'],
            df[df.conversation=='joy_speaker'].groupby('vpn').mean()['pizza'])
plt.show()

print (scipy.stats.spearmanr(final[final.conversation=='joy_speaker'].groupby('vpn').mean()['ZYG'],
            df[df.conversation=='joy_speaker'].groupby('vpn').mean()['pizza']))

plt.scatter(df[df.conversation=='disgust_speaker'].groupby('vpn').mean()['LEV'],
            df[df.conversation=='disgust_speaker'].groupby('vpn').mean()['hering'])
plt.show()

print (scipy.stats.spearmanr(df[df.conversation=='joy_speaker'].groupby('vpn').mean()['LEV'],
            df[df.conversation=='joy_speaker'].groupby('vpn').mean()['hering']))
plt.show()